## This notebook does the following actions: 

 <li>From line shapes of roads, extracts vertices</li>
 <li>Creates buffers from those vertices</li>
 <li>Extracts pixel values to buffers</li>

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import os 
import glob
import pandas as pd
import geopandas as gpd
import numpy as np

In [12]:
from shapely.geometry import LineString, Point
import fiona

In [13]:
import rasterio as rio

In [14]:
# Possible solution to extract vertices from line segments

# list of roads already generated in "Road_visualization.ipynb"

roads = [road for road in glob.glob(os.path.join(r".\vector","st_*.shp"))]




for road in roads:
    list_pts = []# empty list to add coordinates
    # get each of the roads and extract vertices
    print(road)
    with fiona.open(road) as lines:
    # loop through rows (each segment)
        for line in lines:
            st_area_points = Point(line['geometry']['coordinates']) #Creates point from coordinates in linestring

            list_pts.append(st_area_points) # fill with point coordinates

    # Create geopandas with point geometry and save the vertices to file
    
    geodf = gpd.GeoDataFrame( geometry = [coord for coord in list_pts]).to_file(r".\vector\vertices_"+road.split("\\")[-1][3:],driver = "ESRI Shapefile")

.\vector\st_area_1.shp
.\vector\st_area_2.shp
.\vector\st_area_3.shp


## get vertices and buffers

In [15]:
vertices = [vert for vert in glob.glob(os.path.join(r".\vector","vert*.shp"))]

In [16]:
for vert in vertices:
    
    ## Generate the buffers from vertices
    #https://www.earthdatascience.org/courses/use-data-open-source-python/spatial-data-applications/lidar-remote-sensing-uncertainty/extract-data-from-raster/
    #copy of the geodataframe
    buffer_vertices = gpd.read_file(vert).copy()
    
    # 1 m buffer around the points in areas
    buffer_vertices["geometry"] = buffer_vertices.geometry.buffer(1) #update “geometry” column with buffer layer
    buffer_vertices.to_file(r".\vector\buff_"+vert.split("\\")[-1][9:], driver = "ESRI Shapefile")

## extract pixel values

In [17]:
path_stack_4b = r".\raster"
select_stack = [ stk for stk in glob.glob(os.path.join(path_stack_4b,"*STK.tif"))]

In [31]:
buffer_areas =  [buf for buf in glob.glob(os.path.join(r".\vector","buff*.shp"))]

In [35]:
### Define the areas:
areas = ["area_1","area_2","area_3"]

In [19]:
from rasterstats import zonal_stats

## wavelength bands:

col_band = ["b","g","r","nr"] # in order: BLUE, GREEN, RED, NEAR INFRARED

for buffer in buffer_areas:
    
    gdf = gpd.read_file(buffer)
    
    #per area
    for area in areas: 
        
        if area in buffer:
            
            print(area)

            with rio.open(  [ src for src in select_stack if area in src ][0] ) as src:
                
                
                n_bands = src.count # number of bands
                
                transf = src.transform

               

                # multiband iteration
                for band in range(1,n_bands+1): 
                
                    data_array = src.read(band) # get data from origin. Numpy array instead of rasterio data    
            
                    # extract statistics: 3 types
                    stats = zonal_stats(gdf["geometry"], data_array, stats  = ["mean","median","percentile_95"], affine = transf)
            
                    zs_df = pd.DataFrame(stats)
                
                    zs_df.rename(columns={'mean':"mea_"+col_band[band-1],
                              "median":"med"+col_band[band-1],
                              "percentile_95":"p95"+col_band[band-1]}, inplace=True)

                
                    gdf = pd.concat([gdf,zs_df],axis = 1)
                    
            src.close()
            
            display(gdf)
            
            gdf.to_file(os.path.join(r".\statistics","zs_buffers_"+area+".shp"),driver = "ESRI Shapefile")
            
            print("--SHAPEFILE, SAVED--")
            
            print("...")
            
                

area_1


,FID,geometry,mea_b,medb,p95b,mea_g,medg,p95g,mea_r,medr,p95r,mea_nr,mednr,p95nr
0,0,"POLYGON ((659392.300 6473908.700, 659392.295 6...",141.900000,161.0,168.00,157.842857,180.0,189.00,151.428571,177.0,186.55,135.757143,159.5,166.00
1,1,"POLYGON ((659456.300 6473921.900, 659456.295 6...",158.140845,159.0,165.00,172.887324,174.0,179.50,169.732394,171.0,176.00,144.239437,145.0,150.00
2,2,"POLYGON ((659451.200 6473895.800, 659451.195 6...",164.612500,166.5,178.00,180.350000,183.0,193.05,177.062500,177.5,191.00,154.337500,155.5,170.10
3,3,"POLYGON ((659561.300 6473934.000, 659561.295 6...",119.320513,117.5,149.45,131.128205,129.5,169.15,126.576923,125.5,172.30,106.307692,103.0,144.35
4,4,"POLYGON ((659464.800 6473914.000, 659464.795 6...",165.212500,166.0,170.00,179.012500,180.0,183.05,177.000000,178.0,181.05,151.587500,152.5,158.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,390,"POLYGON ((659269.100 6473468.900, 659269.095 6...",169.957746,171.0,176.00,187.042254,188.0,192.00,186.197183,187.0,191.00,165.295775,165.0,170.00
391,391,"POLYGON ((658987.381 6473267.184, 658987.377 6...",116.354430,170.0,174.10,127.189873,188.0,192.00,126.341772,187.0,191.10,114.531646,168.0,172.00
392,392,"POLYGON ((658979.000 6473346.500, 658978.995 6...",69.243590,69.0,73.15,61.717949,62.0,65.15,53.666667,54.0,57.00,34.705128,34.0,39.15
393,393,"POLYGON ((659030.500 6473286.400, 659030.495 6...",108.487179,109.5,134.30,116.615385,118.0,147.45,112.294872,112.0,143.30,92.564103,94.0,123.30


--SHAPEFILE, SAVED--
...
area_2


,FID,geometry,mea_b,medb,p95b,mea_g,medg,p95g,mea_r,medr,p95r,mea_nr,mednr,p95nr
0,0,"POLYGON ((657941.000 6472504.900, 657940.995 6...",126.782051,127.0,150.45,137.076923,136.0,160.15,132.487179,132.0,154.60,107.628205,106.0,130.00
1,1,"POLYGON ((658137.600 6472474.600, 658137.595 6...",168.137500,168.0,172.05,183.637500,184.0,187.05,181.450000,182.0,185.05,161.637500,161.0,167.00
2,2,"POLYGON ((658101.851 6472522.504, 658101.846 6...",106.740260,165.0,177.00,117.077922,185.0,197.20,116.025974,182.0,194.20,105.896104,163.0,173.40
3,3,"POLYGON ((657941.000 6472504.900, 657940.995 6...",126.782051,127.0,150.45,137.076923,136.0,160.15,132.487179,132.0,154.60,107.628205,106.0,130.00
4,4,"POLYGON ((657960.500 6472521.800, 657960.495 6...",123.679487,129.5,161.00,134.153846,144.5,172.60,128.782051,137.0,174.15,110.717949,119.0,155.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,230,"POLYGON ((658674.500 6471822.200, 658674.495 6...",169.076923,170.5,179.00,187.884615,189.0,198.00,186.653846,186.5,196.15,173.448718,174.0,182.15
231,231,"POLYGON ((658698.600 6471809.800, 658698.595 6...",167.562500,169.0,180.00,185.012500,187.0,199.00,182.712500,184.5,196.05,163.325000,166.5,179.00
232,232,"POLYGON ((658674.500 6471822.200, 658674.495 6...",169.076923,170.5,179.00,187.884615,189.0,198.00,186.653846,186.5,196.15,173.448718,174.0,182.15
233,233,"POLYGON ((658758.958 6471860.889, 658758.953 6...",108.181818,180.0,189.00,118.012987,200.0,206.00,116.844156,199.0,204.00,109.480519,183.0,191.00


--SHAPEFILE, SAVED--
...
area_3


,FID,geometry,mea_b,medb,p95b,mea_g,medg,p95g,mea_r,medr,p95r,mea_nr,mednr,p95nr
0,0,"POLYGON ((662729.800 6473800.600, 662729.795 6...",192.687500,192.0,221.00,206.037500,205.0,235.05,202.312500,201.0,231.05,183.800000,180.5,214.10
1,1,"POLYGON ((662630.200 6473790.400, 662630.195 6...",183.875000,164.5,249.00,197.825000,179.0,255.00,192.525000,176.0,255.00,175.212500,154.0,248.05
2,2,"POLYGON ((662620.800 6473794.500, 662620.795 6...",151.794872,150.0,163.30,164.179487,162.0,177.15,161.230769,158.5,174.30,133.538462,131.5,147.45
3,3,"POLYGON ((662630.200 6473790.400, 662630.195 6...",183.875000,164.5,249.00,197.825000,179.0,255.00,192.525000,176.0,255.00,175.212500,154.0,248.05
4,4,"POLYGON ((662624.600 6473803.300, 662624.595 6...",160.397436,159.0,168.15,174.282051,173.0,182.15,171.346154,170.0,180.00,145.256410,144.0,156.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,308,"POLYGON ((662787.400 6473358.200, 662787.395 6...",131.562500,131.0,137.00,141.337500,142.0,146.05,133.687500,134.0,138.00,110.750000,111.0,116.00
309,309,"POLYGON ((662801.500 6473349.600, 662801.495 6...",141.012821,142.5,151.00,150.089744,151.0,162.00,142.871795,144.0,156.00,119.153846,119.0,135.15
310,310,"POLYGON ((662841.132 6473393.929, 662841.127 6...",161.632911,156.0,193.30,174.189873,168.0,207.10,169.139241,163.0,203.20,147.000000,141.0,186.00
311,311,"POLYGON ((662665.800 6473289.500, 662665.795 6...",154.666667,155.0,162.15,165.166667,166.0,172.15,153.384615,154.0,160.15,126.487179,128.0,134.00


--SHAPEFILE, SAVED--
...


In [48]:
path_dsm = r".\raster\dsm"
dsms = [dsm for dsm in glob.glob(os.path.join(path_dsm,"*dsm.tif"))]

In [43]:
buffer_areas =  [buf for buf in glob.glob(os.path.join(r".\vector","buff*.shp"))]

In [66]:

for buff in buffer_areas: 
    print(buff)
    gdf = gpd.read_file(buff)
    gdf.crs = "EPSG:3301"

    #gdf= gdf.to_crs(epsg=3301) # use this projection (EStonian)
    
    for area in areas:

        
        if area in buff:
            
            with rio.open(  [ src for src in dsms if area in src ][0] ) as src:
                
                transf = src.transform
                data_array = src.read(1) # get only one "band". This is the DSM
                src.close()
                
            # Only gets 3 metrics of one band
            stats = zonal_stats(gdf["geometry"], data_array, stats  = ["mean","median","percentile_95"], affine = transf) 
            
            zs_df = pd.DataFrame(stats)
                
            zs_df.rename(columns={'mean':"meaDSM",
                              "median":"medDSM",
                              "percentile_95":"p95DSM"}, inplace=True)

                
            gdf = pd.concat([gdf,zs_df],axis = 1)
            
            display(gdf)
            
            gdf.to_file(os.path.join(r".\statistics","zs_buffer_"+area+"_5vars.shp"),driver = "ESRI Shapefile")
            
            print("--SHAPEFILE, SAVED--")
                
print("finished")

                
                
                
                
                

.\vector\buff_area_1.shp


,FID,geometry,meaDSM,medDSM,p95DSM
0,0,"POLYGON ((659392.300 6473908.700, 659392.295 6...",49.713333,49.610001,49.906999
1,1,"POLYGON ((659456.300 6473921.900, 659456.295 6...",43.113332,43.119999,43.137999
2,2,"POLYGON ((659451.200 6473895.800, 659451.195 6...",45.582497,45.599998,45.887500
3,3,"POLYGON ((659561.300 6473934.000, 659561.295 6...",42.709999,42.514999,43.420000
4,4,"POLYGON ((659464.800 6473914.000, 659464.795 6...",43.090000,43.089996,43.116999
...,...,...,...,...,...
390,390,"POLYGON ((659269.100 6473468.900, 659269.095 6...",58.392498,58.369995,59.784999
391,391,"POLYGON ((658987.381 6473267.184, 658987.377 6...",57.389999,57.389999,57.398998
392,392,"POLYGON ((658979.000 6473346.500, 658978.995 6...",58.035000,58.035000,58.048499
393,393,"POLYGON ((659030.500 6473286.400, 659030.495 6...",56.514999,56.514999,56.519500


--SHAPEFILE, SAVED--
.\vector\buff_area_2.shp


,FID,geometry,meaDSM,medDSM,p95DSM
0,0,"POLYGON ((657941.000 6472504.900, 657940.995 6...",71.167496,71.164993,71.178500
1,1,"POLYGON ((658137.600 6472474.600, 658137.595 6...",70.306661,70.309998,70.309998
2,2,"POLYGON ((658101.851 6472522.504, 658101.846 6...",70.199997,70.199997,70.199997
3,3,"POLYGON ((657941.000 6472504.900, 657940.995 6...",71.167496,71.164993,71.178500
4,4,"POLYGON ((657960.500 6472521.800, 657960.495 6...",73.139999,73.139999,75.327000
...,...,...,...,...,...
230,230,"POLYGON ((658674.500 6471822.200, 658674.495 6...",69.375000,69.375000,70.837500
231,231,"POLYGON ((658698.600 6471809.800, 658698.595 6...",64.186661,63.579998,65.253995
232,232,"POLYGON ((658674.500 6471822.200, 658674.495 6...",69.375000,69.375000,70.837500
233,233,"POLYGON ((658758.958 6471860.889, 658758.953 6...",62.720001,62.720001,62.729000


--SHAPEFILE, SAVED--
.\vector\buff_area_3.shp


,FID,geometry,meaDSM,medDSM,p95DSM
0,0,"POLYGON ((662729.800 6473800.600, 662729.795 6...",55.960002,55.980000,55.988998
1,1,"POLYGON ((662630.200 6473790.400, 662630.195 6...",55.423330,55.430000,55.447997
2,2,"POLYGON ((662620.800 6473794.500, 662620.795 6...",55.494999,55.494999,55.499500
3,3,"POLYGON ((662630.200 6473790.400, 662630.195 6...",55.423330,55.430000,55.447997
4,4,"POLYGON ((662624.600 6473803.300, 662624.595 6...",55.713328,55.709999,55.718998
...,...,...,...,...,...
308,308,"POLYGON ((662787.400 6473358.200, 662787.395 6...",49.063334,49.070000,49.240998
309,309,"POLYGON ((662801.500 6473349.600, 662801.495 6...",48.820000,48.820000,48.820000
310,310,"POLYGON ((662841.132 6473393.929, 662841.127 6...",49.799999,49.799999,49.808498
311,311,"POLYGON ((662665.800 6473289.500, 662665.795 6...",49.524998,49.524998,49.538497


--SHAPEFILE, SAVED--
finished
